<a href="https://colab.research.google.com/github/rakshit-naidu-gt/granite-legal-cookbook/blob/main/recipes/RAG/RAG_over_NH_Caselaw_Summarize_myrun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retrieval Augmented Generation (RAG) using New Hampshire Case Law
*With IBM Granite Models*

The [New Hampshire Case Law Dataset](https://huggingface.co/datasets/free-law/nh) comes from the Caselaw Access Project via Hugging Face.

## In this notebook
This notebook contains instructions for performing Retrieval Augumented Generation (RAG). RAG is an architectural pattern that can be used to augment the performance of language models by recalling factual information from a knowledge base, and adding that information to the model query. The most common approach in RAG is to create dense vector representations of the knowledge base in order to retrieve text chunks that are semantically similar to a given user query.

RAG use cases include:
- Customer service: Answering questions about a product or service using facts from the product documentation.
- Domain knowledge: Exploring a specialized domain (e.g., finance) using facts from papers or articles in the knowledge base.
- News chat: Chatting about current events by calling up relevant recent news articles.

In its simplest form, RAG requires 3 steps:

- Initial setup:
  - Index knowledge-base passages for efficient retrieval. In this recipe, we take embeddings of the passages using WatsonX, and store them in a vector database.
- Upon each user query:
  - Retrieve relevant passages from the database. In this recipe, we using an embedding of the query to retrieve semantically similar passages.
  - Generate a response by feeding retrieved passage into a large language model, along with the user query.

## Prerequisites

To get started, you'll need:
* A [Replicate account](https://replicate.com/) and API token.

## Setting up the environment

### Install dependencies

Granite utils comes with a bundle of dependencies that are required for notebooks.

In [1]:
!pip install --upgrade fsspec==2025.3.0 --quiet
!pip install --upgrade datasets==3.6.0 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 3.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-runtime-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-runtime-cu12 12.5.82 which is incomp

In [2]:
!pip install git+https://github.com/ibm-granite-community/utils.git \
    langchain_community \
    replicate \
    langchain-huggingface \
    langchain-milvus \
    datasets \
    transformers \
    tiktoken --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.8 MB/s eta 0:00:0

## Selecting System Components

### Choose your Embeddings Model

Specify the model to use for generating embedding vectors from text.

To use a model from a provider other than Huggingface, replace this code cell with one from [this Embeddings Model recipe](https://github.com/ibm-granite-community/utils/blob/main/recipes/Components/Langchain_Embeddings_Models.ipynb).

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(model_name="ibm-granite/granite-embedding-30m-english")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/467k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/683 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

### Choose your Vector Database

Specify the database to use for storing and retrieving embedding vectors.

To connect to a vector database other than Milvus, substitute this code cell with one from [this Vector Store recipe](https://github.com/ibm-granite-community/utils/blob/main/recipes/Components/Langchain_Vector_Stores.ipynb).

In [4]:
from langchain_milvus import Milvus
import tempfile

db_file = tempfile.NamedTemporaryFile(prefix="milvus_", suffix=".db", delete=False).name
print(f"The vector database will be saved to {db_file}")

vector_db = Milvus(
    embedding_function=embeddings_model,
    connection_args={"uri": db_file},
    auto_id=True,
    index_params={"index_type": "AUTOINDEX"},
)

The vector database will be saved to /tmp/milvus_bqtbcty8.db


### Choose your LLM
The LLM will be used for answering the question, given the retrieved text.

Follow the instructions in [Getting Started with Replicate](https://github.com/ibm-granite-community/granite-kitchen/blob/cee1513c77429d7ddbf0e5a49b29b7bc9ca0d996/recipes/Getting_Started/Getting_Started_with_Replicate.ipynb), selecting a Granite Code model from the [`ibm-granite`](https://replicate.com/ibm-granite) org.

To connect to a model on a provider other than Replicate, substitute this code cell with one from the [LLM component recipe](https://github.com/ibm-granite-community/granite-kitchen/blob/main/recipes/Components/Langchain_LLMs.ipynb).

In [5]:
from langchain_community.llms import Replicate
from ibm_granite_community.notebook_utils import get_env_var

model_path = "ibm-granite/granite-3.3-8b-instruct"

model = Replicate(
    model=model_path,
    replicate_api_token=get_env_var('REPLICATE_API_TOKEN'),
)

REPLICATE_API_TOKEN not found in Google Colab secrets.
Please enter your REPLICATE_API_TOKEN: ··········


Get the tokenizer used by your chosen model.

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_path)

tokenizer_config.json:   0%|          | 0.00/9.93k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/442k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.48M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/207 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

## Acquiring the Data

We will use a New Hampshire case law dataset to help the model answer questions about NH laws.

### Download the documents

Download the [New Hampshire CAP Caselaw](https://huggingface.co/datasets/free-law/nh) dataset from HuggingFace using the datasets library.

In [7]:
from langchain.document_loaders import HuggingFaceDatasetLoader

# Load the documents from the dataset
loader = HuggingFaceDatasetLoader("free-law/nh", page_content_column="text")
documents = loader.load()
print("Document Count: " + str(len(documents)))

nh.parquet:   0%|          | 0.00/129M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/21540 [00:00<?, ? examples/s]

Document Count: 21540


### Add metadata to the documents

Add the `source` field, which is used below, to the metadata.

In [8]:
for doc in documents:
    doc.metadata['source'] = doc.metadata['id']

### Inspect the documents

In [9]:
for doc in documents[:1]:
    print(doc.metadata, "\n")
    print(doc.page_content, "\n")

{'id': '4439812', 'name': 'Louis C. Wyman v. John A. Durkin Robert L. Stark, Secretary of State Carmen Chimento', 'name_abbreviation': 'Wyman v. Stark', 'decision_date': '1975-01-06', 'docket_number': 'No. 7112', 'first_page': 1, 'last_page': '3', 'citations': '115 N.H. 1', 'volume': '115', 'reporter': 'New Hampshire Reports', 'court': 'New Hampshire Supreme Court', 'jurisdiction': 'New Hampshire', 'last_updated': '2021-08-10T17:25:43.934256+00:00', 'provenance': 'CAP', 'judges': '', 'parties': 'Louis C. Wyman v. John A. Durkin Robert L. Stark, Secretary of State Carmen Chimento', 'head_matter': 'Hillsborough\nNo. 7112\nLouis C. Wyman v. John A. Durkin Robert L. Stark, Secretary of State Carmen Chimento\nJanuary 6, 1975\nStanley M. Brown, Dart S. Bigg, Eugene M. Van Loan III and David R. DePuy (Mr. Brown orally) for the plaintiff.\nDevine, Millimet, Stahl & Branch and Matthias J. Reynolds and William S. Gannon (Mr. Joseph A. Millimet), by brief and orally, for John A. Durkin.\nThomas D

## Building the Document Database

We'll use the caselaw document database to retrieve the full text of the cases by case id.

### Create the database file and document table

In [10]:
# put the json objects in a sqlite database, keyed by id
import sqlite3, os, json

# remove database file if exists
if os.path.isfile('data.db'):
    os.remove('data.db')

conn = sqlite3.connect('data.db')
c = conn.cursor()

# create the table if it doesn't exist. include id, text, and size
c.execute('''CREATE TABLE IF NOT EXISTS data
             (id INTEGER PRIMARY KEY UNIQUE,
              metadata TEXT,
              text TEXT,
              char_count INTEGER)''')


### Insert the documents into the table

In [11]:
for doc in documents:
    id = doc.metadata["id"]
    c.execute("INSERT INTO data (id, metadata, text, char_count) VALUES (?,?,?,?)", (id, json.dumps(doc.metadata), doc.page_content, doc.metadata["char_count"]))
    conn.commit()

### Count the documents

In [12]:
c.execute("SELECT count(*) FROM data")
doc_count = c.fetchone()[0]
print(f"Document count: {doc_count}")

Document count: 21540


## Building the Vector Database

In this example, we take the caselaw text, split it into chunks, derive embedding vectors using the embedding model, and load it into the vector database for querying.

### Split the document into chunks

Split the document into text segments that can fit into the model's context window.

In [13]:
from langchain.text_splitter import TokenTextSplitter

# Split the documents into chunks
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=10)
chunks = text_splitter.split_documents(documents)
print("Chunk Count: " + str(len(chunks)))

Chunk Count: 55959


### Inspect the chunks

In [14]:
import json
for i in range(1):
    print(chunks[i].page_content)
    print(json.dumps(chunks[i].metadata, indent=4))

"Per curiam.\nThis transfer arises out of the same case which was the subject matter of the petition for writ of prohibition in Durkin v. Hillsborough County Superior Court, 114 N.H. 788, 330 A.2d 777 (1974). The Superior Court (Bois, J.) has transferred without ruling seven questions, the first of which is as follows: \u201cDoes the Superior Court have jurisdiction either through RSA 68:4 II; other jurisdictional statutes or through precedent, to invalidate an election for United States Senator?\u201d\nThe several States may regulate the conduct of senatorial elections and may provide procedures necessary to guard against irregularity and error in the tabulation of votes and against fraud and corrupt practices. U.S. Const. art. I, \u00a7 4; Smiley v. Holm, 285 U.S. 355 (1932). They may provide procedures for a recount so long as they do not impair or frustrate the Senate\u2019s ability to make an independent judgment. Roudebush v. Hartke, 405 U.S. 15 (1972).\nThe proceedings before th

### Populate the vector database

NOTE: Population of the vector database may take a few minutes depending on your embedding model and service.

In [ ]:
ids = vector_db.add_documents(chunks)
print("Document IDs: " + str(ids[:3]))

## Querying the Databases

### Create query text

Here we use a topic of NH law to query into the vector database for relevant cases. Because we will consider one case at a time (due to context length restrictions), phrase the query to consider a single case.

In [ ]:
query = "Summarize this court case about the Suspension and Expulsion of Pupils, using the IRAC framework (Issue, Rule, Application, Conclusion).\n\n"

### Query the vector database

Query the vector database for cases related to the law. Similar documents are found by proximity of the embedded vector in vector space.

In [ ]:
k = 10  # the number of docs to retrieve
docs_with_score = vector_db.similarity_search_with_score(query, k=k)

# Get a unique set of docs.
docs = []
doc_ids = {}
for doc, score in docs_with_score:
    # print(doc.metadata["name_abbreviation"])
    # print(score)
    id = doc.metadata["id"]
    if id not in doc_ids:
        docs.append(doc)
        print(id, " - ", doc.metadata["name_abbreviation"])
        doc_ids[id] = 1

### Query the document database

Get the full text of the first case found by the vector search.

Get a list of unique doc ids.

In [ ]:
# Get a list of unique doc ids.
docs_ids_seen = set()
uq_docs = [doc for doc in docs if not (doc.metadata["id"] in docs_ids_seen or docs_ids_seen.add(doc.metadata["id"]))]

In [ ]:
# Retrieve a number of cases.
cases = []

for doc in docs:
    case_id = doc.metadata["id"]
    case_short_name = doc.metadata["name_abbreviation"]

    c.execute("SELECT text FROM data where id = ?", (case_id,))
    case_text = c.fetchone()[0]
    case_length = len(tokenizer.tokenize(case_text))

    # For this recipe, only consider cases that can fit in the 4k context window (along with the 512 token output).
    if case_length < 3500:
        cases.append((case_id, case_short_name, case_text))
        print(f"Case Id: {case_id}")
        print(f"Case Name: {case_short_name}")
        print(f"Case Length: {case_length} tokens\n")


## Answering Questions

### Assemble the Chat Prompt

Build a chat prompt template with the law and the retrieved case.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

system_prompt = (
    "You are an assistant with legal expertise. Answer the question based only on the following text from a NH court case. Do not include any other court cases. \n\n{case_text}"
)

rag_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

rag_chain = (
    rag_prompt
    | model
    | StrOutputParser()
)

### Ask questions of the retrieved case in relation to the law.

Answer the question about each related case.

In [ ]:
for case in cases[:2]:
    (case_id, case_short_name, case_text) = case
    response = rag_chain.invoke(input = {"input": query, "case_text": case_text})
    print(f"Case {case_id}: {case_short_name}\n")
    print(response, "\n\n")